In [2]:
# reformat (tall->wide) 7 PAH EPR 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, time

## Reformat EPR data

In [19]:
complete_input_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/7_PAH/01_11_2021/input/tall/7_PAH_zf_EPR_data_2021JAN11.csv'
behav_all_data = pd.read_csv(complete_input_file_path, header = 0)

In [4]:
behav_all_data.head()

,chemical.id,bottle.id,conc,plate.id,well,variable,value
0,3762,C20062,0.0,21269,H01,t1,0.0
1,3762,C20062,0.0,21269,H02,t1,0.0
2,3762,C20062,0.0,21269,H03,t1,0.0
3,3762,C20062,0.0,21269,H04,t1,0.0
4,3762,C20062,0.0,21269,H05,t1,0.0


In [6]:
behav_all_data = behav_all_data.dropna()

In [12]:
#behav_all_data_TX = behav_all_data.loc[behav_all_data['bottle.id'] == "TX002271",['chemical.id', 'bottle.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']]
#behav_all_data_TX.head()

In [7]:
# Keep only relevant columns
behav_all_data = behav_all_data.rename(columns = {"variable":"endpoint"})
columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
behav_all_data_select = behav_all_data.loc[:,columns_to_keep]
behav_all_data_select.head()

,chemical.id,conc,plate.id,well,endpoint,value
0,3762,0.0,21269,H01,t1,0.0
1,3762,0.0,21269,H02,t1,0.0
2,3762,0.0,21269,H03,t1,0.0
3,3762,0.0,21269,H04,t1,0.0
4,3762,0.0,21269,H05,t1,0.0


In [27]:
display(np.unique(behav_all_data_select['endpoint']))
display(len(np.unique(behav_all_data_select['endpoint'])))

array(['t1', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17',
       't18', 't19', 't2', 't20', 't21', 't22', 't23', 't24', 't25',
       't26', 't27', 't28', 't29', 't3', 't30', 't31', 't32', 't33',
       't34', 't35', 't36', 't37', 't38', 't39', 't4', 't40', 't41',
       't42', 't43', 't44', 't45', 't46', 't47', 't48', 't49', 't5',
       't50', 't6', 't7', 't8', 't9'], dtype=object)

50

In [30]:
start_time = time.time()

reformatted_data = pd.DataFrame()
max_time = 50
#full_devel = "full"
full_devel = "devel"

if (full_devel == "full"):
    # all chemicals
    chemical_id_from_here = np.unique(behav_all_data['chemical.id'])
else: # full_devel = "devel"
    chemical_id_from_here = np.unique([3762])

for chemical_index in chemical_id_from_here:
    behav_data_chemical = behav_all_data_select.loc[behav_all_data['chemical.id'] == chemical_index,:]
    print("chemical_index:" + str(chemical_index))
    # Append chemical_plate_well as a unique identifier
    behav_data_chemical.insert(0, 'chemical_plate_well', behav_data_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    
    for cpw in np.unique(behav_data_chemical.chemical_plate_well):
        temp_df = behav_data_chemical.loc[behav_data_chemical.chemical_plate_well == cpw,:]
        #display(temp_df.head())
        temp_df_grouped = temp_df.groupby(['chemical.id', 'plate.id', 'well'])
        # display(temp_df_grouped.head())
        for name, group in temp_df_grouped:
            # print ("len(group.endpoint):" + str(len(group.endpoint)))
            if(len(group.endpoint) == max_time):
                temp = pd.DataFrame(
                        {
                        'chemical.id': np.unique(temp_df['chemical.id']),
                        'plate.id': np.unique(temp_df['plate.id']),
                        'well': np.unique(temp_df['well']),
                        'chemical_plate_well': np.unique(temp_df['chemical_plate_well']),
                        'conc': np.unique(temp_df['conc'])
                        })
                #print(temp.head())
                # Append additonal columns corresponding to time points
                for time_point in np.arange(max_time):
                    end_point = 't'+ str(time_point+1)
                    temp = pd.concat([temp, pd.DataFrame({end_point: temp_df.value[temp_df.endpoint == end_point].values})],axis = 1)
                #print(temp.head())
                #print(reformatted_data)
                reformatted_data = pd.concat([reformatted_data, temp])
                #print(reformatted_data)
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("Done, it took:"+str(time_took)) 
# took 13 minutes for 1 chemical

chemical_index:3762
Done, it took:13.1 seconds


In [31]:
display(reformatted_data)

,chemical.id,plate.id,well,chemical_plate_well,conc,t1,t2,t3,t4,t5,...,t41,t42,t43,t44,t45,t46,t47,t48,t49,t50
0,3762,21269,A01,3762_21269_A01,100.0,0.000000,24.956625,2.126737,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
0,3762,21269,A02,3762_21269_A02,100.0,6.749119,0.000000,0.000000,20.941841,0.000000,...,0.000000,0.000000,0.000000,57.421894,0.000000,0.000000,0.0,0.0,0.0,0.000000
0,3762,21269,A03,3762_21269_A03,100.0,0.000000,28.689219,19.661444,0.000000,0.000000,...,0.000000,0.000000,0.000000,37.890635,0.021701,0.000000,0.0,0.0,0.0,0.000000
0,3762,21269,A04,3762_21269_A04,100.0,0.000000,38.368058,0.000000,0.000000,31.228287,...,0.000000,0.000000,0.000000,0.000000,0.000000,31.532106,0.0,0.0,0.0,38.281251
0,3762,21269,A05,3762_21269_A05,100.0,0.000000,0.000000,0.000000,0.000000,24.175362,...,16.124142,0.000000,0.000000,18.294272,0.000000,54.861156,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,3762,21295,H08,3762_21295_H08,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.430553,1.605903,8.940969,7.378483,0.000000,0.000000,0.0,0.0,0.0,0.000000
0,3762,21295,H09,3762_21295_H09,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
0,3762,21295,H10,3762_21295_H10,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
0,3762,21295,H11,3762_21295_H11,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


In [32]:
reformatted_data_filename = str(complete_input_file_path[:-4]) + "_wide_t1_t50_" + str(full_devel) + ".csv"
reformatted_data.to_csv(reformatted_data_filename,index=False)